In [3]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#general imports
import sys
sys.path.append('..')
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import time

In [6]:
#notebook specific imports
from utils.artifact_removal import artifact_removal
from utils.utils_funcs import threshold_detect, stim_start_frame, paq_data
import tifffile
from utils.paq2py import paq_read
import copy


In [7]:
#global plotting params
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
sns.set()
sns.set_style('white')

In [8]:
# tiff_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001/2019-04-25_RL036_t-001_Cycle00001_Ch3.tif'
# naparm_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_NAPARM'
# paq_path = r'/home/jamesrowland/Documents/packerstation/rlees/Data/2019-04-25/RL036/2019-04-25_RL036_t-001.paq'

In [9]:
tiff_path = '/home/jamesrowland/Documents/packerstation/rlees/Data/2019-03-19/RL025/2019-03-19_RL025_t-007/2019-03-19_RL025_t-007_Cycle00001_Ch3.tif'
naparm_path = '/home/jamesrowland/Documents/packerstation/rlees/Data/2019-03-19/RL025/NAPARM/2019-03-19_RL025_NAPARM_003'
paq_path= '/home/jamesrowland/Documents/packerstation/rlees/Data/2019-03-19/RL025/2019-03-19_RL025_photostim_003.paq'

In [10]:
stack = tifffile.imread(tiff_path)
stack = stack[0:500,:,:]
stack_original = copy.deepcopy(stack)
stack.shape

(500, 512, 512)

In [11]:
paq = paq_read(paq_path)
stim_frames = stim_start_frame(paq, 'markpoints2packio')
frame_clock = paq_data(paq, 'frame_clock', threshold_ttl=True)
stim_frames

[182641,
 202724,
 222810,
 242893,
 262977,
 283063,
 303146,
 323232,
 342646,
 362729,
 382815,
 402899,
 422985,
 443068,
 463152,
 483238,
 502652,
 522735,
 542822,
 562905,
 582989,
 603075,
 623159,
 643242,
 662659,
 682742,
 702826,
 722912,
 742996,
 763080,
 783166,
 803250,
 822664,
 842750,
 862834,
 882918,
 903001,
 923088,
 943172,
 963255,
 982669,
 1002756,
 1022840,
 1042924,
 1063007,
 1083094,
 1103178,
 1123262,
 1142676,
 1162762,
 1182846,
 1202930,
 1223014,
 1243098,
 1263184,
 1283268,
 1302682,
 1322766,
 1342850,
 1362937,
 1383021,
 1403105,
 1423189,
 1443273,
 1462690,
 1482774,
 1502858,
 1522942,
 1543026,
 1563112,
 1583196,
 1603280,
 1622695,
 1642779,
 1662863,
 1682949,
 1703033,
 1723117,
 1743202,
 1762616,
 1782700,
 1802787,
 1822871,
 1842955,
 1863039,
 1883123,
 1903207,
 1922624,
 1942708,
 1962792,
 1982876,
 2002960,
 2023045,
 2043129,
 2063213,
 2082630,
 2102714,
 2122798,
 2142882,
 2162966,
 2183051,
 2203135,
 2223219,
 2242636,
 

In [12]:
duration_ms = 250 #length of stim
duration_samples = (duration_ms / 1000) * paq['rate']

In [13]:
#use this to remove frames during stim based on paq
to_remove = []

for stim_frame in stim_frames: 

    # frame indexes that fall during the stim
    in_stim = np.where((frame_clock >= stim_frame) & (frame_clock <= stim_frame + duration_samples))[0]
    
    #empircal observation, these are the frames with artifact
    in_stim = np.append(in_stim, in_stim[-1]+1)
    in_stim = np.append(in_stim, in_stim[-1]+1)
    
    to_remove.append(in_stim)

to_remove = np.ravel(to_remove)

In [14]:
# stack[to_remove, :, :] = 0
# tifffile.imwrite('test.tiff', stack)

In [15]:
t1 = time.time()

to_remove = [i for i in to_remove if i < stack.shape[0]]

stack_removed = artifact_removal(stack, remove_me=to_remove, width_thresh=5)

t2 = time.time()

t2-t1

72.60886240005493

In [16]:

tifffile.imwrite('test_removed.tiff', stack_removed)
tifffile.imwrite('test.tiff', stack_original)

In [92]:
len(to_remove)

130

In [93]:
to_remove 

[115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 474,
 475,
 476,
 477,
 478,
 479,
 480,
 481,
 482,
 483]

In [ ]:
from artifact_removal import artifact_removal
import tifffile

stack = tifffile.imread(path_to_stack)
remove_me = list_of_frames_with_artifact_in

cleaned_stack = artifact_removal(stack, remove_me=remove_me, width_thresh=5)


In [94]:
(60/500) * 1000

120.0